In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os


# --- CONFIGURACIÓN ---
DATA_PATH = 'dataset_final_powerbi.csv' 
OUTPUT_DIR = 'eda_results'

if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

sns.set_theme(style="whitegrid", palette="muted")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = ['DejaVu Sans'] 

# --- CARGA Y PREPARACIÓN DE DATOS ---
try:
    df = pd.read_csv(DATA_PATH, encoding='utf-8-sig')
except Exception as e:
    print(f"Error al cargar el archivo: {e}")
    raise

df['Brecha de Experiencia'] = pd.to_numeric(df['Brecha de Experiencia'], errors='coerce')
df['Índice de Compatibilidad Total'] = pd.to_numeric(df['Índice de Compatibilidad Total'], errors='coerce')
df['Cantidad_Habilidades'] = pd.to_numeric(df['Cantidad_Habilidades'], errors='coerce')

df.dropna(subset=['Brecha de Experiencia', 'Índice de Compatibilidad Total', 'Cantidad_Habilidades'], inplace=True)

# --- FUNCIONES DE ANÁLISIS ---

def get_top_items(series, top_n=10):
    all_items = []
    for item_list in series.dropna():
        items = [item.strip().title() for item in item_list.split(',') if item.strip()]
        all_items.extend(items)
    item_counts = pd.Series(all_items).value_counts()
    return item_counts.head(top_n)

def generate_text_frequency_analysis(df, col):
    top_items = get_top_items(df[col])
    
    plt.figure(figsize=(10, 6))
    # CORRECCIÓN: Se añade hue=top_items.index y legend=False para evitar el warning
    sns.barplot(
        x=top_items.values, 
        y=top_items.index, 
        hue=top_items.index, 
        palette="viridis", 
        legend=False
    ) 
    plt.title(f'Top {len(top_items)} {col}', fontsize=16)
    plt.xlabel('Frecuencia', fontsize=12)
    plt.ylabel(col, fontsize=12)
    plt.tight_layout()
    
    filename = f'top_{col.lower().replace(" ", "_").replace("á", "a").replace("é", "e").replace("í", "i").replace("ó", "o").replace("ú", "u")}.png'
    plt.savefig(os.path.join(OUTPUT_DIR, filename))
    plt.close()
    print(f"✅ Gráfico de frecuencia para '{col}' guardado.")

# --- EJECUCIÓN ---
text_cols = ['Habilidades Técnicas', 'Objetivos Profesionales', 'Necesidades de Networking', 'Propuesta de Valor']
for col in text_cols:
    generate_text_frequency_analysis(df, col)

# --- ANÁLISIS BIVARIADO ---

# 1. Brecha de Experiencia vs. Compatibilidad
plt.figure(figsize=(10, 6))
sns.boxplot(x='Brecha de Experiencia', y='Índice de Compatibilidad Total', data=df, color='skyblue')
plt.title('Índice de Compatibilidad Total por Brecha de Experiencia', fontsize=16)
plt.xlabel('Brecha de Experiencia', fontsize=12)
plt.ylabel('Índice de Compatibilidad Total', fontsize=12)
plt.savefig(os.path.join(OUTPUT_DIR, 'compatibilidad_vs_brecha_experiencia.png'))
plt.close()

# 2. Modalidad de Trabajo vs. Compatibilidad
plt.figure(figsize=(8, 6))
# CORRECCIÓN: Se añade hue='Modalidad de Trabajo' y legend=False
sns.boxplot(
    x='Modalidad de Trabajo', 
    y='Índice de Compatibilidad Total', 
    data=df, 
    hue='Modalidad de Trabajo', 
    palette='Set2', 
    legend=False
)
plt.title('Índice de Compatibilidad Total por Modalidad de Trabajo', fontsize=16)
plt.xlabel('Modalidad de Trabajo', fontsize=12)
plt.ylabel('Índice de Compatibilidad Total', fontsize=12)
plt.savefig(os.path.join(OUTPUT_DIR, 'compatibilidad_vs_modalidad_trabajo.png'))
plt.close()

# 3. Cantidad de Habilidades vs. Compatibilidad
plt.figure(figsize=(10, 6))
sns.regplot(x='Cantidad_Habilidades', y='Índice de Compatibilidad Total', data=df, 
            scatter_kws={'alpha':0.1, 'color':'#e74c3c'}, line_kws={'color':'#2c3e50'})
plt.title('Relación: Cantidad de Habilidades vs. Índice de Compatibilidad Total', fontsize=16)
plt.xlabel('Cantidad de Habilidades', fontsize=12)
plt.ylabel('Índice de Compatibilidad Total', fontsize=12)
plt.savefig(os.path.join(OUTPUT_DIR, 'compatibilidad_vs_cantidad_habilidades.png'))
plt.close()

print("\n--- Análisis Completado sin advertencias de paleta ---")


✅ Gráfico de frecuencia para 'Habilidades Técnicas' guardado.
✅ Gráfico de frecuencia para 'Objetivos Profesionales' guardado.
✅ Gráfico de frecuencia para 'Necesidades de Networking' guardado.
✅ Gráfico de frecuencia para 'Propuesta de Valor' guardado.

--- Análisis Completado sin advertencias de paleta ---
